In [5]:

import os
import numpy as np
import pandas as pd
import openslide
from skimage.filters import threshold_otsu
from skimage.morphology import remove_small_objects
from skimage.measure import label, regionprops
from scipy.spatial import KDTree
from scipy import stats
import gc
import warnings
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import json
from datetime import datetime
import traceback
warnings.filterwarnings("ignore")